# Why Hat
> Analytical approach using Neural Network on tabulate data

The core engine for project $\large\hat{y}$.

The dataset is from [New York City Airbnb Open Data](https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data#AB_NYC_2019.csv) on kaggle

In [1]:
# default_exp whyhat

In [80]:
# export
import pandas as pd
import numpy as np
from pathlib import Path
import os
import json
from torchember.core import color

In [6]:
DATA = Path("../data")

In [7]:
CSV_PATH = DATA/"AB_NYC_2019.csv"

In [9]:
df = pd.read_csv(CSV_PATH)
df.sample(100)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
18722,14823077,Amazing Private Bedroom on UES of New York City,20707006,Tianna,Manhattan,East Harlem,40.78962,-73.94837,Private room,100,4,1,2017-01-03,0.03,1,0
20052,16056111,A Touch of Modern,36420662,Marc,Brooklyn,Williamsburg,40.70775,-73.94611,Private room,75,3,3,2017-04-29,0.11,1,0
14905,11837962,1 BR heart of Midtown East,61391963,Corporate Housing,Manhattan,Midtown,40.75660,-73.96956,Entire home/apt,133,30,7,2018-09-29,0.22,91,127
26622,21175822,Studio in Upper East Side Manhattan,152885643,Seamus,Manhattan,Upper East Side,40.77585,-73.95101,Entire home/apt,128,4,12,2018-12-26,0.60,1,0
5096,3662887,1 Bedroom Apartment in Gramcery!,14041891,Tom,Manhattan,Flatiron District,40.74110,-73.98455,Entire home/apt,200,4,0,NaN,NaN,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35471,28137157,"Laurelton, Queens, NYC (3min-JFK; Seconds-LIRR)",157393936,Apostle John,Queens,Springfield Gardens,40.66959,-73.75246,Private room,100,5,1,2018-09-13,0.10,2,365
6489,4717440,"Bright, art-filled elevator apt, LES/Chinatown",3237577,Eddie,Manhattan,Chinatown,40.71585,-73.98982,Entire home/apt,170,3,94,2019-06-23,1.71,1,7
48347,36207836,Big Room close to LGA Airport. Cuarto Grande.,157053317,Jey,Queens,East Elmhurst,40.76263,-73.88999,Private room,40,1,0,NaN,NaN,1,318
29595,22715051,Duplex Apt in Brooklyn Beautiful Brownstone,24967136,Paul,Brooklyn,Bedford-Stuyvesant,40.68226,-73.93153,Entire home/apt,119,5,25,2019-05-25,1.49,2,77


### Config how we learn the columns

In [122]:
# export
class RichColumn(object):
    """
    A pandas series manager
    """
    def __init__(self,column,min_occur = 5):
        self.col = column
        self.col.rc = self
        self.name = self.col.name
        self.min_occur = min_occur
        self.use = True
        self.is_conti = True
        self.defined = False
        
    def kill(self):
        self.defined = True
        self.use = False
        
    def conti(self):
        self.defined = True
        self.is_conti = True
        
    def disc(self):
        self.defined = True
        self.is_conti = False
        
    def is_number(self):
        return self.col.dtype in (int,float,
                              np.float16,np.float32,np.float64,np.float64,
                              np.int0,np.int8,np.int16,np.int32,np.int64)
    
    def __bool__(self):
        return self.use
    
    def __len__(self):
        if self.is_conti:
            return 1
        else:
            width = len(self.top_freq)
            width =1 if width==2 else width
            return width
    
    def __repr__(self,):
        return f"<Rich Column:{self.name}>"
    
    def top_freq_(self):
        freq = self.freq()
        self.top_freq = freq[freq[self.name]>=self.min_occur].reset_index()
        return self.top_freq
    
    def freq(self):
        return pd.DataFrame(data=self.col.value_counts())
    
class RichDF(object):
    """
    A pandas dataframe manager
    """
    def __init__(self,df):
        self.df = df
        self.columns = dict()
        for colname in self.df:
            self.columns.update({colname:RichColumn(df[colname])})
        
    @property
    def col_conf(self):
        return dict((k,{"use":v.use,"is_cont":v.is_conti}) for k,v in self.columns.items())
        
    def kill(self,colname):
        self.df[colname].rc.kill()
        
    def conti(self,colname):
        self.df[colname].rc.conti()
        
    def disc(self,colname):
        self.df[colname].rc.disc()
        
    def set_col(self,rcol):
        if rcol.defined:
            print(f"{rcol.name} defined, use:{rcol.use}, contineus?:{rcol.is_conti}")
        print(color.bold("="*30))
        print(color.cyan(rcol.name))
        print(color.red(f"number? {rcol.is_number()}"))
        print(rcol.top_freq_().head(5))
              
        print(color.red("Is this a [C]ontineous, [D]iscrete or a column we do[N]'t need? default N"))
        x = input().lower()
        if x=="c":
            rcol.conti()
    
        elif x =="d":
            rcol.disc()
            
        elif x =="":
            rcol.kill()
            
        elif x=="n":
            rcol.kill()
            
        else:
            print(color.yellow(f"{x} not found Again?"))
        
        
    def tour(self):
        for colname in self.df:
            col = self.df[colname]
            while col.rc.defined==False:
                self.set_col(col.rc)
                
            

In [123]:
rdf = RichDF(df)

In [125]:
rdf.tour()